In [42]:
import json
import xgboost as xgb
from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import random

from numpy import mean
from numpy import std
import pandas as pd
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense


from preprocessing import for_train

2022-10-04 00:23:59.783790: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [37]:
f_train = open("../../data/train.json", 'r')
train_data = json.load(f_train)

def find_discard_authors(data, p):
    # p = 0.20250
    # p = -1

    n_samples = len(data)
    empty_idx = []
    if p == -1:
        return []

    for i in tqdm(range(n_samples), desc="delet some useless data"):
        authors = data[i]['authors']
        p_author = 0

        for au in authors:
            if au < 100:
                p_author += 1
        
        if p_author == 0:
            empty_idx.append(i)
    
    print("Number of instance with label : ", n_samples-len(empty_idx))
    
    remain = int((p*(n_samples-len(empty_idx))/(1-p)))
    
    print("Number of instance without label(remain) : ", remain)

    return random.sample(empty_idx, len(empty_idx) - remain)

def get_author_matrix(data, discard_idx=[], train=True):
    n_samples = len(data)

    # get prolific authors 
    # co author -> feature / 单独编码
    # 多任务 - predict all authors 


    if train:
        y = np.ndarray([n_samples-len(discard_idx), 100])
        key = 'authors'
    else:
        y = None

    # get co-author matrix
    if train:
        amatrix = np.ndarray([n_samples-len(discard_idx), 21245 - 100 + 1])
    else:
        amatrix = np.ndarray([n_samples, 21245 - 100 + 1])
        key = 'coauthors'

    INDEX = 0
    for i in tqdm(range(n_samples), desc="authors"):
        if i in discard_idx and train:
            continue

        authors = data[i][key]
        
        for au in authors:
            if au < 100:
                y[INDEX, au] += 1

            else:
                amatrix[INDEX, au - 100] += 1

        INDEX += 1
        
    return amatrix, y

def get_year_venue_matrix(data, discard_idx=[], train=True):
    n_samples = len(data)

    # get venue feature
    # embedding

    if train:
        vmatrix = np.ndarray([n_samples-len(discard_idx), 466])
    else:
        vmatrix = np.ndarray([n_samples, 466])

    INDEX = 0
    for i in tqdm(range(n_samples), desc="venue"):
        if i in discard_idx and train:
            continue

        venue = data[i]['venue']
        
        if venue:
            vmatrix[INDEX, venue] = 1
        else:
            vmatrix[INDEX, 465] = 1
        INDEX += 1

    if train:
        ymatrix = np.ndarray([n_samples-len(discard_idx), 20])
    else:
        ymatrix = np.ndarray([n_samples, 20])
    

    INDEX = 0
    for i in tqdm(range(n_samples), desc="year"):
        if i in discard_idx and train:
            continue

        year = data[INDEX]['year']
        
        if year:
            ymatrix[INDEX, year] = 1
        else:
            ymatrix[INDEX, year] = 0
        INDEX += 1

    return np.concatenate((vmatrix, ymatrix), 1)


di = find_discard_authors(train_data, p=0.20250)
_, y = get_author_matrix(train_data, di)
X = get_year_venue_matrix(train_data, di)

delet some useless data: 100%|██████████| 25793/25793 [00:00<00:00, 901350.42it/s]


Number of instance with label :  7460
Number of instance without label(remain) :  1894


year: 100%|██████████| 25793/25793 [00:04<00:00, 6064.80it/s]


In [46]:
# y_1 = y[:, 1:5]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

# X_train = X
# X_test = X
# y_train = y
# y_test = y

print("Train:")
print("     X_train : ", X_train.shape)
print("     y_train : ", y_train.shape)
print("Test_Kaggle:")
print("     X_test  : ", X_test.shape)
print("     y_test  : ", y_test.shape)


Train:
     X_train :  (7483, 486)
     y_train :  (7483, 100)
Test_Kaggle:
     X_test  :  (1871, 486)
     y_test  :  (1871, 100)


In [43]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]

    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

    for train_ix, test_ix in cv.split(X):

        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]

        model = get_model(n_inputs, n_outputs)

        model.fit(X_train, y_train, verbose=0, epochs=100)

        yhat = model.predict(X_test)
        yhat = yhat.round()
        acc = accuracy_score(y_test, yhat)

        print('>%.3f' % acc)
        results.append(acc)

    return results

In [47]:
from numpy import asarray

n_inputs, n_outputs = X.shape[1], y.shape[1]

model = get_model(n_inputs, n_outputs)

model.fit(X_train, y_train, verbose=0, epochs=100)

In [48]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, recall_score, f1_score

y_pred_train = model.predict(X_train)
y_pred = model.predict(X_test)

59/59 [==============================] - 0s 943us/step


In [58]:
def multi_label_column(matrix):
    
    n_samples, n_class = matrix.shape
    # print(n_samples, n_class)

    output =[]

    for i in range(n_samples):
        pred = ""
        for j in range(n_class):
            if matrix[i][j] >= 0.1:
                pred += str(j) + " "
        if pred:
            output.append(pred[:-1])
        else:
            output.append("-1")
    
    return output

In [59]:
y_test.shape

(1871, 100)

In [60]:
y_train_list = multi_label_column(y_train)
y_pred_train_list = multi_label_column(y_pred_train)

y_test_list = multi_label_column(y_test)
y_pred_list = multi_label_column(y_pred)

In [61]:
print('='*25 + 'Evaluation results' + '='*25)
print('The accuracy score of prediction is: {}'.format(accuracy_score(y_test_list, y_pred_list)))
print('The racall score of prediction is: {}'.format(recall_score(y_test_list, y_pred_list, average='weighted')))
print('The f1 score of prediction is: {}'.format(f1_score(y_test_list, y_pred_list, average='weighted'))) 

=========================Evaluation results=========================
The accuracy score of prediction is: 0.15713522180652056
The racall score of prediction is: 0.15713522180652056
The f1 score of prediction is: 0.07426359420270866


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
y_pred_train

array([[9.1210119e-03, 6.7151412e-03, 8.5109333e-03, ..., 2.0335540e-02,
        1.0140832e-02, 1.0872827e-02],
       [3.8836230e-02, 2.5586510e-02, 6.6189835e-04, ..., 2.3895851e-03,
        1.1077715e-03, 2.5966848e-04],
       [1.3430017e-02, 1.0919531e-02, 4.4666328e-03, ..., 4.8015043e-03,
        3.0004121e-03, 2.7487981e-03],
       ...,
       [6.6489474e-06, 2.8531868e-03, 3.0807371e-03, ..., 7.3693544e-03,
        1.0027222e-02, 6.6739781e-04],
       [4.2460905e-03, 4.4979141e-03, 1.1782616e-02, ..., 1.1800103e-02,
        6.0533406e-03, 2.2307139e-02],
       [5.7770931e-03, 5.2451496e-03, 1.0570733e-02, ..., 1.0764746e-02,
        4.2453292e-03, 1.7640496e-02]], dtype=float32)

In [39]:
# create XGBoost instance with default hyper-parameters
xgb_estimator = xgb.XGBClassifier(objective='binary:logistic')

# create MultiOutputClassifier instance with XGBoost model inside
multilabel_model = OneVsRestClassifier(xgb_estimator)

# fit the model
multilabel_model.fit(X_train, y_train)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None,
                                            reg_alpha=None, reg_lambda=None, ...))

In [41]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

y_pred = multilabel_model.predict(X_test)

print("accuracy : ", accuracy_score(y_test, y_pred))
print("recall   : ", recall_score(y_test, y_pred, average='weighted'))
print("f1       : ", f1_score(y_test, y_pred, average='weighted'))

accuracy :  0.9690005344735435
recall   :  0.017543859649122806
f1       :  0.03070175438596491


In [24]:
X_train

array([[0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [1.89240875e-052, 4.90750276e-062, 1.78800382e-052, ...,
        0.00000000e+000, 1.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       ...,
       [4.57672550e-072, 5.15146484e-062, 1.18051609e-095, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [1.18014677e-095, 2.99939715e-067, 1.20253480e+285, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [9.50277535e-043, 1.08651609e-095, 4.57668662e-072, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000]])

In [25]:
X_, y_ = make_multilabel_classification(n_samples=3000, n_features=45, n_classes=20, n_labels=1,
                                      allow_unlabeled=False, random_state=42)

In [30]:
y_train.shape

(7483, 100)

In [36]:
y_

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 1]])